# Connery Toolkit

Using this toolkit, you can integrate Connery Actions into your LangChain agents and chains.

If you want to use only one particular Connery Action in your agent,
check out the [Connery Action Tool](/docs/integrations/tools/connery) documentation.

## What is Connery?

Connery is an open-source plugin infrastructure for AI.

With Connery, you can easily create a custom plugin with a set of actions and seamlessly integrate them into your LangChain agents and chains. 
Connery will handle the rest: runtime, authorization, secret management, access management, audit logs, and other vital features.
Additionally, Connery and our community offer a wide range of ready-to-use open-source plugins for your convenience.

Learn more about Connery:

- GitHub: https://github.com/connery-io/connery-platform
- Documentation: https://docs.connery.io

## Prerequisites

To use Connery Actions in your LangChain agents, you need to do some preparation:

1. Set up the Connery runner using the [Quickstart](https://docs.connery.io/docs/platform/quick-start/) guide.
2. Install all the plugins with the actions you want to use in your agent.
3. Set environment variables `CONNERY_RUNNER_URL` and `CONNERY_RUNNER_API_KEY` so the toolkit can communicate with the Connery Runner.

## Example

In the example below, we create an agent that uses two Connery Actions to summarize a public webpage and send the summary by email:

1. **Summarize public webpage** from the [Summarization](https://github.com/connery-io/summarization-plugin) plugin.
2. **Send email** from the [Gmail](https://github.com/connery-io/gmail) plugin.

NOTE: Connery Action is a structured tool, so you can only use it in the agents supporting structured tools.

In [3]:
import os

from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent_toolkits.connery import ConneryToolkit
from langchain.chat_models import ChatOpenAI
from langchain.tools.connery import ConneryService

# Specify your Connery Runner credentials.
os.environ["CONNERY_RUNNER_URL"] = ""
os.environ["CONNERY_RUNNER_API_KEY"] = ""

# Specify OpenAI API key.
os.environ["OPENAI_API_KEY"] = ""

# Specify your email address to receive the email with the summary from example below.
recepient_email = "test@example.com"

connery_service = ConneryService()
connery_toolkit = ConneryToolkit.create_instance(connery_service)

In [4]:
llm = ChatOpenAI(temperature=0)
agent = initialize_agent(
    connery_toolkit.get_tools(), llm, AgentType.OPENAI_FUNCTIONS, verbose=True
)

prompt = f"""Make a short summary of the webpage http://www.paulgraham.com/vb.html in three sentences
and send it to {recepient_email}. Include the link to the webpage into the body of the email."""
agent.run(prompt)



> Entering new AgentExecutor chain...

Invoking: `CA72DFB0AB4DF6C830B43E14B0782F70` with `{'publicWebpageUrl': 'http://www.paulgraham.com/vb.html'}`


{'summary': 'The author reflects on the shortness of life and how it affects our priorities. They discuss how having children helps them understand the finite nature of time and the importance of prioritizing meaningful experiences. The author emphasizes the need to eliminate unnecessary and fake aspects of life and focus on what truly matters. They also discuss the detrimental effects of online arguments and the need to be conscious of how we spend our time. The author suggests actively seeking out meaningful experiences and not waiting to pursue them. They also mention the importance of savoring the present moment and avoiding the rush of daily tasks. The author concludes by urging readers to prune out the unnecessary, prioritize what matters, and make the most of the time they have.'}
Invoking: `CABC80BB79C15067CA983495324AE709` wit

'I have sent the email with the summary of the webpage to test@example.com. Please check your inbox.'